In [3]:
#imports
import pandas as pd 
import numpy as np
import pickle
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.feature_extraction import text
import datetime
import ast
import sys

In [4]:
sys.path.insert(0, '../../data_tools/')
from clean_text import stopwords_make, punctstr_make, unicode_make, clean_sentence
stop_words_list = stopwords_make()
unicode_list = unicode_make()
punctstr = punctstr_make()

In [11]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

In [6]:
data = pd.read_pickle('../../nowdata/charters_2015.pkl')

In [7]:
#data['WEBTEXT'] = data['WEBTEXT'].apply(ast.literal_eval)

In [8]:
data.loc[0,"WEBTEXT"][2][3]

'icator Dashboard\nOur goal is to create 75,000 credentials/degrees by 2030. This can only happen when the entire community is purposeful through data in establishing and enabling conditions for long-term attainment success.\nCCI Dashboard Sponsored in part by the J.L. Bedsole Foundation\nHome\n | \nAbout\n | \nOur Work\n | \nOur Impact\n | \nNews/ Media\n | \nSupport\n | \nContact\n© 2018 Copyright Mobile Area Education Foundation.  All rights reserved. | Site by \n Web and New Media\n'

In [9]:
data['WEBTEXT'] = data['WEBTEXT'].fillna('0')

In [10]:
#webtext = data["WEBTEXT"]
#webtext_with_text = []
#for t in webtext:
#    try:
#        len_site = len(t)
#        for i in range(len_site):
#            webtext_with_text.append(t[i][3])
#    except:
#        pass

webtext = data["WEBTEXT"]
webtext_with_text = []
tokens = []
for t in webtext:
    try:
        len_site = len(t)
        #initializing a master string
        school_string = ''
        for i in range(len_site):
            school_string += t[i][3] + ' \n '
        webtext_with_text.append(school_string)
    except:
        pass

In [ ]:
'''
# creating custom stop words

my_additional_stop_word_list = []
#long list of manual addition to stopwords:
for i in range(1,13):
    my_additional_stop_word_list.append(datetime.date(2008, i, 1).strftime('%B'))
for i in range(1,13):
    my_additional_stop_word_list.append((datetime.date(2008, i, 1).strftime('%B')).lower())
for i in range(1, 2100):
    my_additional_stop_word_list.append(str(i))
    
my_additional_stop_word_list.append('00')
my_additional_stop_word_list.append('el')
my_additional_stop_word_list.append('en')
my_additional_stop_word_list.append('la')
my_additional_stop_word_list.append('los')
my_additional_stop_word_list.append('para')
my_additional_stop_word_list.append('las')
my_additional_stop_word_list.append('san')
my_additional_stop_word_list.append('mr')
my_additional_stop_word_list.append('mrs')
my_additional_stop_word_list.append('sa')
my_additional_stop_word_list.append('angeles')
my_additional_stop_word_list.append('diego')
my_additional_stop_word_list.append('california')
my_additional_stop_word_list.append('york')
my_additional_stop_word_list.append('fax')
my_additional_stop_word_list.append('email')
my_additional_stop_word_list.append('phone')
my_additional_stop_word_list.append('harlem')
my_additional_stop_word_list.append('bronx')
my_additional_stop_word_list.append('wi')
my_additional_stop_word_list.append('pm')
my_additional_stop_word_list.append('org')
my_additional_stop_word_list.append('city')
my_additional_stop_word_list.append('austin')
my_additional_stop_word_list.append('antonio')
my_additional_stop_word_list.append('texas')

#adding states abbreviations for stopwords
states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV"]
for state in states:
          my_additional_stop_word_list.append(state)
for state in [state.lower() for state in states]:
          my_additional_stop_word_list.append(state)
        
stop_words_extra = text.ENGLISH_STOP_WORDS.union(my_additional_stop_word_list)
'''

In [12]:
# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
no_features = 1000
tf_vectorizer = CountVectorizer(max_df=0.5, min_df=0.02, max_features=no_features, stop_words=stop_words_list)
tf = tf_vectorizer.fit_transform(webtext_with_text)
tf_feature_names = tf_vectorizer.get_feature_names()

/opt/conda/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['breakfast', 'calendar', 'carolina', 'columbia', 'contact', 'dakota', 'district', 'facebook', 'faq', 'faqs', 'hampshire', 'island', 'jersey', 'lunch', 'menu', 'mexico', 'new', 'north', 'rhode', 'south', 'us', 'west', 'york'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [14]:
# Define no. of topics
no_topics = 20
# Run LDA
lda = LatentDirichletAllocation(n_components=no_topics, max_iter=5, learning_method='batch', learning_offset=50.,random_state=43).fit(tf)

In [15]:
#old version
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print ("Topic " + str(topic_idx) + ': ' + " ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))
no_top_words = 20
display_topics(lda, tf_feature_names, no_top_words)
#topics to note: topic 1, topic 2, topic 4, topic , topic 15

Topic 0: website reserved rights west academy university corporation presence schoolmessenger information connections high college please grade state board policy page curriculum
Topic 1: arts high college course science year program charter university academy class music art parent community english history dance news information
Topic 2: college prep high campus enroll career state enrollment university program new public schools info events board learn support lake work
Topic 3: city value schools high summit read charter central news university year new board hs hill south teachers community work learning
Topic 4: grade day week class math please year calendar science read 8th time first reading 6th kindergarten child parent learning teacher
Topic 5: style size text family class lead color strong times new stem top doctorate http level type yes fit data center
Topic 6: montessori elementary services community county home center children district academy schools child view program i

In [16]:
pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)

/opt/conda/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
17    -0.452765 -0.003345       1        1  19.100176
4      0.049234  0.041030       2        1   7.984563
18    -0.000319 -0.106843       3        1   6.203904
9      0.018995 -0.094185       4        1   6.062487
14     0.060029  0.008983       5        1   6.024910
13     0.045707  0.025985       6        1   5.944106
10     0.051566 -0.028659       7        1   5.508243
1      0.033481 -0.020272       8        1   5.122403
15     0.042079  0.029045       9        1   4.742734
19     0.061795  0.041418      10        1   4.096361
8      0.009970  0.040135      11        1   3.992965
16     0.010572 -0.045548      12        1   3.885289
2      0.046178 -0.038434      13        1   3.871115
12     0.040291  0.024078      14        1   3.648245
11     0.072807  0.007317      15        1   3.169805
6      0.050348  0.017959      16        1   3.161198
7     -0.144661 -0.070105      17        1   2.634272
0      0.059267  0.021591      18        1   2.628844
3      0.024929 -0.049696      19        1   1.844099
5     -0.079505  0.199545      20        1   0.374280, topic_info=    Category          Freq         Term         Total  loglift  logprob
453  Default  2.689000e+05         idea  2.689000e+05  30.0000  30.0000
783  Default  1.032913e+06      schools  1.032913e+06  29.0000  29.0000
157  Default  7.466630e+05      charter  7.466630e+05  28.0000  28.0000
814  Default  1.007350e+05         size  1.007350e+05  27.0000  27.0000
439  Default  2.901530e+05   homeschool  2.901530e+05  26.0000  26.0000
508  Default  7.720150e+05     learning  7.720150e+05  25.0000  25.0000
845  Default  2.319140e+05         stem  2.319140e+05  24.0000  24.0000
615  Default  3.981600e+05       online  3.981600e+05  23.0000  23.0000
168  Default  2.586150e+05        class  2.586150e+05  22.0000  22.0000
27   Default  4.345830e+05      academy  4.345830e+05  21.0000  21.0000
281  Default  2.041150e+05     dyslexia  2.041150e+05  20.0000  20.0000
333  Default  1.521410e+05       family  1.521410e+05  19.0000  19.0000
181  Default  3.696250e+05      college  3.696250e+05  18.0000  18.0000
860  Default  1.743480e+05      success  1.743480e+05  17.0000  17.0000
430  Default  4.259790e+05         high  4.259790e+05  16.0000  16.0000
400  Default  3.745860e+05        grade  3.745860e+05  15.0000  15.0000
126  Default  3.064530e+05        board  3.064530e+05  14.0000  14.0000
166  Default  7.690700e+04         city  7.690700e+04  13.0000  13.0000
241  Default  1.740230e+05         data  1.740230e+05  12.0000  12.0000
681  Default  9.954700e+04         prep  9.954700e+04  11.0000  11.0000
570  Default  1.715720e+05       method  1.715720e+05  10.0000  10.0000
141  Default  2.297810e+05     calendar  2.297810e+05   9.0000   9.0000
470  Default  2.945770e+05  information  2.945770e+05   8.0000   8.0000
271  Default  4.352600e+04    doctorate  4.352600e+04   7.0000   7.0000
269  Default  1.725040e+05     district  1.725040e+05   6.0000   6.0000
665  Default  2.091580e+05       please  2.091580e+05   5.0000   5.0000
596  Default  2.645860e+05          new  2.645860e+05   4.0000   4.0000
183  Default  4.813700e+04        color  4.813700e+04   3.0000   3.0000
563  Default  1.424010e+05      meeting  1.424010e+05   2.0000   2.0000
290  Default  1.634440e+05   elementary  1.634440e+05   1.0000   1.0000
..       ...           ...          ...           ...      ...      ...
924  Topic20  1.943203e+03         type  2.307637e+04   3.1134  -4.6469
325  Topic20  1.340521e+03         face  1.631668e+04   3.0888  -5.0182
521  Topic20  1.377132e+03         line  1.941208e+04   2.9420  -4.9913
120  Topic20  1.286024e+03        black  2.131609e+04   2.7800  -5.0597
333  Topic20  7.945489e+03       family  1.521419e+05   2.6357  -3.2387
271  Topic20  2.237671e+03    doctorate  4.352669e+04   2.6200  -4.5058
905  Topic20  2.264106e+03       

## Try 2

In [ ]:
no_features = 1000
# creating custom stop words
my_additional_stop_word_list = []

'''
#long list of manual addition to stopwords:
for i in range(1,13):
    my_additional_stop_word_list.append(datetime.date(2008, i, 1).strftime('%B'))
for i in range(1,13):
    my_additional_stop_word_list.append((datetime.date(2008, i, 1).strftime('%B')).lower())
for i in range(1, 2100):
    my_additional_stop_word_list.append(str(i))
my_additional_stop_word_list.append('00')
my_additional_stop_word_list.append('el')
my_additional_stop_word_list.append('en')
my_additional_stop_word_list.append('la')
my_additional_stop_word_list.append('los')
my_additional_stop_word_list.append('para')
my_additional_stop_word_list.append('las')
my_additional_stop_word_list.append('san')
my_additional_stop_word_list.append('mr')
my_additional_stop_word_list.append('mrs')
my_additional_stop_word_list.append('sa')
my_additional_stop_word_list.append('angeles')
my_additional_stop_word_list.append('diego')
my_additional_stop_word_list.append('california')
my_additional_stop_word_list.append('york')
my_additional_stop_word_list.append('fax')
my_additional_stop_word_list.append('email')
my_additional_stop_word_list.append('phone')
my_additional_stop_word_list.append('harlem')
my_additional_stop_word_list.append('bronx')
my_additional_stop_word_list.append('wi')
my_additional_stop_word_list.append('pm')
my_additional_stop_word_list.append('org')
my_additional_stop_word_list.append('city')
my_additional_stop_word_list.append('austin')
my_additional_stop_word_list.append('antonio')
my_additional_stop_word_list.append('texas')
#adding states abbreviations for stopwords
states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV"]
for state in states:
          my_additional_stop_word_list.append(state)
for state in [state.lower() for state in states]:
          my_additional_stop_word_list.append(state)
stop_words_extra = text.ENGLISH_STOP_WORDS.union(my_additional_stop_word_list)
'''

# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
tf_vectorizer = CountVectorizer(max_df=0.5, min_df=0.02, max_features=no_features, stop_words=stop_words_extra)
tf = tf_vectorizer.fit_transform(webtext_with_text)
tf_feature_names = tf_vectorizer.get_feature_names()

In [ ]:
# No. of topics - 10
no_topics = 10
# Run LDA
lda = LatentDirichletAllocation(n_topics=no_topics, max_iter=5, learning_method='batch', learning_offset=50.,random_state=0).fit(tf)

In [ ]:
#old version
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print ("Topic " + str(topic_idx) + ': ' + " ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))
no_top_words = 20
display_topics(lda, tf_feature_names, no_top_words)

In [ ]:
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)